<a href="https://colab.research.google.com/github/benjamininden/AI-teaching-python/blob/main/AIMLChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An AIML based chatbot

Chatbots can use a variety of techniques to generate responses to user input. A common one is based on rules where each rule consists of a pattern to be matched by the input, and an output that is generated in this case. [Artificial Intelligence Markup Language (AIML)](http://www.aiml.foundation/doc.html) allows to specify those rules in XML format such that interpreters are available for executing those rules.

The code below provides an example of a simple AIML based chatbot with connections to Wikipedia an Open Weathermap as extras. This chatbot happens to be specialised on the topic of urban agriculture. Suggested execises:

*   Modify the chatbot to talk about your favourite topic.
*   Ultimately, pattern matching is a bit inflexible against variations in word order, typing errors, etc. Another approach is to calculate the similarities between the user input and a long list of possible user input. The answer linked to the possible input that is most similar to the actual input is then returned. For this to work, one can convert a sentence into a feature dictionary using the bag of words technique, filter out stop words, and create feature vectors from the result using the tf-idf technique. Similarity between two feature vectors can be calculated using cosine similarity. You can add this approach to the chatbot, the individual methods are implemented within the [scikit-learn](https://scikit-learn.org/stable/) library and you find more information about them in the documentation.

For our basic chatbot, we may first want to install a few Python packages that are missing from most Python installations.

In [2]:
! pip3 install -q wikipedia-api
! pip3 install -q aiml

Next, we create the XML file. Note that there are different kinds of rules, some refer to other rules using the srai tag. Also, some rules return their original input preceded by somthing like "#0$". This signals to the Python program to use another method to (further) process the input.

In [3]:
%%writefile mybot.xml
<aiml version="1.0.1" encoding="UTF-8">

<!-- Greetings and Politeness -->
<category> <pattern> HELLO </pattern>
  <template>Hello! How are you?</template> </category>

<category> <pattern> HELLO * </pattern>
  <template><srai>HELLO</srai></template> </category>

<category> <pattern> HI </pattern>
  <template><srai>HELLO</srai></template> </category>

<category> <pattern> HI * </pattern>
  <template><srai>HELLO</srai></template> </category>

<category> <pattern> HOW ARE YOU </pattern>
  <template>I am functioning within normal parameters, thanks for asking.</template></category>

<category><pattern> I AM FINE </pattern>
  <template>That is good to know. What would you like to discuss regarding urban agriculture?</template></category>

<category><pattern> BYE </pattern>
  <template>#0$Bye! Nice talking to you. You take care now.</template></category>

<category><pattern> GOODBYE </pattern>
  <template><srai>BYE</srai></template></category>

<category><pattern> EXIT </pattern>
  <template><srai>BYE</srai></template></category>

<category> <pattern> THANKS </pattern>
  <template>You are welcome.</template></category>

<category> <pattern> THANK YOU </pattern>
  <template><srai>THANKS</srai></template></category>

<category> <pattern> CHEERS </pattern>
  <template>Sure mate.</template></category>

<category> <pattern> CHEERS * </pattern>
  <template>Sure mate.</template></category>

<!-- Weather API -->
<category><pattern> WHAT IS THE WEATHER LIKE IN * </pattern>
  <template>#2$<star/></template></category>

<category><pattern> HOW IS THE WEATHER IN * </pattern>
  <template><srai>WHAT IS THE WEATHER LIKE IN <star/></srai></template></category>

<!--- Wikipedia API -->
<category><pattern> WHAT IS A * </pattern>
  <template>#1$<star index="1"/></template></category>

<category><pattern> WHAT IS AN * </pattern>
  <template><srai>WHAT IS A <star/></srai></template></category>

<category><pattern> WHAT IS * </pattern>
  <template><srai>WHAT IS A <star/></srai></template></category>

<category><pattern> DEFINE * </pattern>
  <template><srai>WHAT IS A <star/></srai></template></category>

<category><pattern> WHO IS * </pattern>
  <template><srai>WHAT IS A <star/></srai></template></category>

<!-- Default reaction -->
<category><pattern> * </pattern>
  <template>#99$<star index="1"/></template></category>

</aiml>


Writing mybot.xml


We need to initialise the Wikipedia-API.

In [4]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')
wikipediaapi.log.setLevel(level=wikipediaapi.logging.ERROR)

Same for the OpenWeathermap API.

In [5]:
import json, requests
APIkey = "" #insert your personal OpenWeathermap API key here if you have one, and want to use this feature

And here is the initialisation of the AIML interpreter.

In [6]:
import aiml
kern = aiml.Kernel()
kern.setTextEncoding(None)
kern.bootstrap(learnFiles="mybot.xml")

Loading mybot.xml...done (0.00 seconds)
Kernel bootstrap completed in 0.00 seconds


Finally, the welcoming message and main loop of the program. This is were post-processing of the AIML interpreter output is done. Once you want to end the conversation, be polite and say good bye properly and the program will let you go.

In [8]:
print("Welcome to the urban agriculture chat bot. Please feel free to ask questions about",
      "concepts and methods in making your garden a food production site,  Permaculture,",
      "Aquaponics, crops, the weather, or any plant images you might have.")
while True:
    #get user input
    try:
        userInput = input("> ")
    except (KeyboardInterrupt, EOFError) as e:
        print("Bye!")
        break
    #pre-process user input and determine response agent (if needed)
    responseAgent = 'aiml'
    #activate selected response agent
    if responseAgent == 'aiml':
        answer = kern.respond(userInput)
    #post-process the answer for commands
    if answer[0] == '#':
        params = answer[1:].split('$')
        cmd = int(params[0])
        if cmd == 0:
            print(params[1])
            break
        elif cmd == 1:
            wpage = wiki_wiki.page(params[1])
            if wpage.exists():
                print(wpage.summary)
                print("Learn more at", wpage.canonicalurl)
            else:
                print("Sorry, I don't know what that is.")
        elif cmd == 2:
            succeeded = False
            api_url = r"http://api.openweathermap.org/data/2.5/weather?q="
            response = requests.get(api_url + params[1] + r"&units=metric&APPID="+APIkey)
            if response.status_code == 200:
                response_json = json.loads(response.content)
                if response_json:
                    t = response_json['main']['temp']
                    tmi = response_json['main']['temp_min']
                    tma = response_json['main']['temp_max']
                    hum = response_json['main']['humidity']
                    wsp = response_json['wind']['speed']
                    wdir = response_json['wind']['deg']
                    conditions = response_json['weather'][0]['description']
                    print("The temperature is", t, "°C, varying between", tmi, "and", tma, "at the moment, humidity is", hum, "%, wind speed ", wsp, "m/s,", conditions)
                    succeeded = True
            if not succeeded:
                print("Sorry, I could not resolve the location you gave me.")
        elif cmd == 99:
            print("I did not get that, please try again.")
    else:
        print(answer)

Welcome to the urban agriculture chat bot. Please feel free to ask questions about concepts and methods in making your garden a food production site,  Permaculture, Aquaponics, crops, the weather, or any plant images you might have.
> how are you
I am functioning within normal parameters, thanks for asking.
> what is a cucumber
Cucumber (Cucumis sativus) is a widely-cultivated creeping vine plant in the Cucurbitaceae gourd family that bears usually cylindrical fruits, which are used as vegetables. Considered an annual plant, there are three main varieties of cucumber — slicing, pickling, and burpless/seedless — within which several cultivars have been created. The cucumber originates from South Asia, but now grows on most continents, as many different types of cucumber are traded on the global market. In North America, the term wild cucumber refers to plants in the genera Echinocystis and Marah, though the two are not closely related.
Learn more at https://en.wikipedia.org/wiki/Cucumbe